## Configuration

In [1]:
### Configuration
# Dependencies
import os
import pandas as pd
import numpy as np
from summerplaylist import *
from datetime import date

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

# Old Playlists
playlistsOld = [
    'Summer 2018',
    'Summer 2019',
    'Endless Summer 2020',
    'Endless Summer 2021',
    'Endless Summer 2022',
    'Endless Summer 2023', 
    'Endless Summer 2024'
]

# Filepaths
artistsKnownPath = os.path.join('data', 'artistsKnown.csv')
artistsHistoricalPath = os.path.join('data', 'artistsHistorical.csv')

## Log In

In [2]:
# Authenticate to Spotify
credentials = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Read in Known Artists

In [3]:
# Read in known artists from csv
if os.path.exists(artistsKnownPath):
    artistsKnown = pd.read_csv(artistsKnownPath)
else:
    artistsKnown = pd.DataFrame(columns=['artistID', 'following'])

artistsKnown.head()
print(len(artistsKnown))

3106


## Read in old playlists

In [4]:
# Empty df
artistsOld = pd.DataFrame()
# Read in all old summer playlists
for playlistName in playlistsOld:
    # Get playlistID
    id_pl = getPlaylistID(credentials, playlistName)
    # Get playlistInfo
    tracks_pl = fetchTracksFromPlaylist(credentials, id_pl)
    # Append to df
    artistsOld = pd.concat([artistsOld, tracks_pl], axis=0)
# Deduplicate
artistsOld = artistsOld.drop_duplicates('artistID', ignore_index=True)[['artistID']]

print(len(artistsOld))
artistsOld.head()


1096


,artistID
0,2ZRQcIgzPCVaT9XKhXZIzh
1,3ApUX1o6oSz321MMECyIYd
2,44eNlYXYkIGob3NU82tgLq
3,3mJ9GlkLzj8Ka7Z7EQaCMi
4,2cCUtGK9sDU2EoElnk0GNB


## Merge and deduplicate known artists and any from new playlists

In [5]:
artistsKnown = pd.concat([artistsKnown, artistsOld], axis=0).drop_duplicates('artistID', ignore_index=True)
# Mark Liked and Unliked artists
artistsKnown.loc[artistsKnown['artistID'].isin(artistsOld['artistID']), 'following'] = True
artistsKnown.loc[~artistsKnown['artistID'].isin(artistsOld['artistID']), 'following'] = False
artistsKnown

,artistID,artistName,artistPopularity,artistFollowers,artistGenre,date,following
0,1URnnhqYAYcrqrcwql10ft,21 Savage,89.0,21260129.0,"atl hip hop, hip hop, rap",2025-01-04,True
1,2RTUTCvo6onsAnheUk3aL9,3 Doors Down,73.0,4573400.0,"alternative metal, nu metal, post-grunge, rock",2025-01-04,False
2,41Q0HrwWBtuUkJc7C1Rp6K,311,63.0,1102549.0,"alternative metal, alternative rock, funk meta...",2025-01-04,False
3,4NiJW4q9ichVqL1aUsgGAN,A Day To Remember,70.0,2418808.0,"alternative metal, metalcore, neon pop punk, p...",2025-01-04,False
4,09hVIj6vWgoCDtT03h8ZCa,A Tribe Called Quest,68.0,2247820.0,"conscious hip hop, east coast hip hop, golden ...",2025-01-04,False
...,...,...,...,...,...,...,...
3103,7m0BsF0t3K9WQFgKoPejfk,ArrDee,63.0,1036092.0,uk hip hop,2025-01-04,True
3104,3IZkE54DC1Pk1dbsU0J0Pk,Gizzle,8.0,36.0,NaN,2025-01-04,True
3105,4tMWqWMZ05588dPKYgCwYC,Pinch,16.0,12.0,NaN,2025-01-04,True
3106,6IFr4XgzsnDsErOLZ6fAAb,NaN,NaN,NaN,NaN,NaN,True


## Update all artists' info based on Spotify's records and add it to historical records

In [6]:
# Get artist info based on list of IDs
artistInfo = fetchArtistsInfo(credentials, artistsKnown['artistID'].to_list())
# Set the date of the data capture
artistInfo['date'] = date.today()
# Record whether artist was liked or not
artistInfo = artistInfo.merge(artistsKnown['artistID'], how='left', on='artistID')

# Read in historical
# Read in known artists from csv
if os.path.exists(artistsHistoricalPath):
    artistsHistorical = pd.read_csv(artistsHistoricalPath)
else:
    artistsHistorical = pd.DataFrame(columns=['date', 'following', 'artistName', 'artistID', 'artistPopularity','artistFollowers', 'artistGenre'])

# Merge in new data
artistsHistorical = pd.concat([artistsHistorical, artistInfo], axis=0, ignore_index=True)

artistsHistorical.head()
print(len(artistsHistorical))

19799


## Load

In [7]:
# Write artistsKnown and artistsHistorical csvs
artistInfo.to_csv(artistsKnownPath, index=False)
artistsHistorical.to_csv(artistsHistoricalPath, index=False)